In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

In [2]:
data=pd.read_csv("UrbanSound8K/metadata/UrbanSound8K.csv")

In [3]:
data.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [4]:
data["fold"].value_counts()

4     990
5     936
3     925
2     888
1     873
7     838
10    837
6     823
9     816
8     806
Name: fold, dtype: int64

In [6]:
from librosa import display
import librosa

In [11]:
x_train=[]
x_test=[]
y_train=[]
y_test=[]
path="UrbanSound8K/audio/fold"
for i in tqdm(range(len(data))):
    fold_no=str(data.iloc[i]["fold"])
    file=data.iloc[i]["slice_file_name"]
    label=data.iloc[i]["classID"]
    filename=path+fold_no+"/"+file
    y,sr=librosa.load(filename)
    mfccs = np.mean(librosa.feature.mfcc(y, sr, n_mfcc=40).T,axis=0)
    melspectrogram = np.mean(librosa.feature.melspectrogram(y=y, sr=sr, n_mels=40,fmax=8000).T,axis=0)
    chroma_stft=np.mean(librosa.feature.chroma_stft(y=y, sr=sr,n_chroma=40).T,axis=0)
    chroma_cq = np.mean(librosa.feature.chroma_cqt(y=y, sr=sr,n_chroma=40).T,axis=0)
    chroma_cens = np.mean(librosa.feature.chroma_cens(y=y, sr=sr,n_chroma=40).T,axis=0)
    features=np.reshape(np.vstack((mfccs,melspectrogram,chroma_stft,chroma_cq,chroma_cens)),(40,5))
    if(fold_no!='10'):
      x_train.append(features)
      y_train.append(label)
    else:
      x_test.append(features)
      y_test.append(label)

  1%|▌                                                                             | 67/8732 [01:01<2:19:47,  1.03it/s]c:\users\kartik kumar\appdata\local\programs\python\python37\lib\site-packages\librosa\core\pitch.py:146: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')
100%|████████████████████████████████████████████████████████████████████████████| 8732/8732 [1:46:00<00:00,  3.01it/s]


In [12]:
x_train=np.array(x_train)
x_test=np.array(x_test)
y_train=np.array(y_train)
y_test=np.array(y_test)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((7895, 40, 5), (837, 40, 5), (7895,), (837,))

In [13]:
x_train_2d=np.reshape(x_train,(x_train.shape[0],x_train.shape[1]*x_train.shape[2]))
x_test_2d=np.reshape(x_test,(x_test.shape[0],x_test.shape[1]*x_test.shape[2]))
x_train_2d.shape,x_test_2d.shape

((7895, 200), (837, 200))

In [14]:
np.savetxt("train_data.csv", x_train_2d, delimiter=",")
np.savetxt("test_data.csv",x_test_2d,delimiter=",")
np.savetxt("train_labels.csv",y_train,delimiter=",")
np.savetxt("test_labels.csv",y_test,delimiter=",")

In [15]:
#extracting data from csv files into numpy arrays
from numpy import genfromtxt
x_train = genfromtxt('train_data.csv', delimiter=',')
y_train = genfromtxt('train_labels.csv', delimiter=',')
x_test = genfromtxt('test_data.csv', delimiter=',')
y_test = genfromtxt('test_labels.csv', delimiter=',')

In [16]:
#shape
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((7895, 200), (837, 200), (7895,), (837,))

In [17]:
#converting to one hot
from keras.utils.np_utils import to_categorical
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)
y_train.shape,y_test.shape

Using TensorFlow backend.


((7895, 10), (837, 10))

In [18]:

#reshaping to 2D 
x_train=np.reshape(x_train,(x_train.shape[0], 40,5))
x_test=np.reshape(x_test,(x_test.shape[0], 40,5))
x_train.shape,x_test.shape

((7895, 40, 5), (837, 40, 5))

In [19]:
#reshaping to shape required by CNN
x_train=np.reshape(x_train,(x_train.shape[0], 40,5,1))
x_test=np.reshape(x_test,(x_test.shape[0], 40,5,1))

In [20]:

#shapes
x_train.shape,x_test.shape

((7895, 40, 5, 1), (837, 40, 5, 1))

In [21]:
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Dropout

In [22]:
#forming model
model=Sequential()

W1014 20:30:18.206816  1592 deprecation_wrapper.py:119] From c:\users\kartik kumar\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



In [23]:
#adding layers and forming the model
model.add(Conv2D(64,kernel_size=5,strides=1,padding="Same",activation="relu",input_shape=(40,5,1)))
model.add(MaxPooling2D(padding="same"))

model.add(Conv2D(128,kernel_size=5,strides=1,padding="same",activation="relu"))
model.add(MaxPooling2D(padding="same"))
model.add(Dropout(0.3))

model.add(Flatten())

model.add(Dense(256,activation="relu"))
model.add(Dropout(0.3))

model.add(Dense(512,activation="relu"))
model.add(Dropout(0.3))

model.add(Dense(10,activation="softmax"))

W1014 20:31:05.111484  1592 deprecation_wrapper.py:119] From c:\users\kartik kumar\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1014 20:31:05.339161  1592 deprecation_wrapper.py:119] From c:\users\kartik kumar\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1014 20:31:05.470014  1592 deprecation_wrapper.py:119] From c:\users\kartik kumar\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W1014 20:31:05.488005  1592 deprecation_wrapper.py:119] From c:\users\kartik kumar\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_de

In [24]:
#compiling
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

W1014 20:31:13.622036  1592 deprecation_wrapper.py:119] From c:\users\kartik kumar\appdata\local\programs\python\python37\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1014 20:31:13.642980  1592 deprecation_wrapper.py:119] From c:\users\kartik kumar\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [25]:
#training the model
model.fit(x_train,y_train,batch_size=50,epochs=30,validation_data=(x_test,y_test))

W1014 20:31:20.093037  1592 deprecation.py:323] From c:\users\kartik kumar\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 7895 samples, validate on 837 samples
Epoch 1/30
7895/7895 [==============================] - ETA: 3:11 - loss: 3.3331 - acc: 0.160 - ETA: 1:40 - loss: 3.5518 - acc: 0.130 - ETA: 1:09 - loss: 3.4039 - acc: 0.140 - ETA: 53s - loss: 3.2731 - acc: 0.150 - ETA: 44s - loss: 3.1257 - acc: 0.15 - ETA: 38s - loss: 3.1168 - acc: 0.15 - ETA: 34s - loss: 3.0152 - acc: 0.17 - ETA: 30s - loss: 2.9559 - acc: 0.17 - ETA: 28s - loss: 2.8889 - acc: 0.17 - ETA: 25s - loss: 2.8442 - acc: 0.16 - ETA: 24s - loss: 2.8286 - acc: 0.16 - ETA: 22s - loss: 2.7782 - acc: 0.16 - ETA: 21s - loss: 2.7397 - acc: 0.16 - ETA: 20s - loss: 2.6961 - acc: 0.17 - ETA: 19s - loss: 2.6602 - acc: 0.17 - ETA: 18s - loss: 2.6472 - acc: 0.17 - ETA: 17s - loss: 2.6269 - acc: 0.17 - ETA: 17s - loss: 2.6091 - acc: 0.17 - ETA: 16s - loss: 2.5937 - acc: 0.16 - ETA: 16s - loss: 2.5785 - acc: 0.16 - ETA: 15s - loss: 2.5617 - acc: 0.16 - ETA: 15s - loss: 2.5405 - acc: 0.16 - ETA: 14s - loss: 2.5368 - acc: 0.16 - ETA: 14s - loss:

7895/7895 [==============================] - ETA: 8s - loss: 0.9419 - acc: 0.700 - ETA: 8s - loss: 0.9625 - acc: 0.690 - ETA: 8s - loss: 1.1332 - acc: 0.626 - ETA: 8s - loss: 1.0851 - acc: 0.640 - ETA: 8s - loss: 1.0473 - acc: 0.656 - ETA: 8s - loss: 1.0758 - acc: 0.643 - ETA: 8s - loss: 1.0433 - acc: 0.651 - ETA: 8s - loss: 1.0570 - acc: 0.647 - ETA: 8s - loss: 1.0762 - acc: 0.633 - ETA: 8s - loss: 1.0724 - acc: 0.636 - ETA: 8s - loss: 1.0684 - acc: 0.638 - ETA: 8s - loss: 1.0533 - acc: 0.641 - ETA: 7s - loss: 1.0569 - acc: 0.636 - ETA: 7s - loss: 1.0705 - acc: 0.638 - ETA: 7s - loss: 1.0841 - acc: 0.633 - ETA: 7s - loss: 1.0810 - acc: 0.632 - ETA: 7s - loss: 1.0916 - acc: 0.624 - ETA: 7s - loss: 1.1048 - acc: 0.621 - ETA: 7s - loss: 1.1106 - acc: 0.616 - ETA: 7s - loss: 1.1069 - acc: 0.616 - ETA: 7s - loss: 1.1112 - acc: 0.615 - ETA: 7s - loss: 1.1148 - acc: 0.611 - ETA: 7s - loss: 1.1050 - acc: 0.619 - ETA: 7s - loss: 1.1049 - acc: 0.619 - ETA: 7s - loss: 1.1096 - acc: 0.615 - ETA: 

7895/7895 [==============================] - ETA: 8s - loss: 0.8202 - acc: 0.760 - ETA: 8s - loss: 0.8546 - acc: 0.700 - ETA: 8s - loss: 0.7749 - acc: 0.740 - ETA: 8s - loss: 0.7931 - acc: 0.735 - ETA: 9s - loss: 0.7775 - acc: 0.736 - ETA: 10s - loss: 0.8087 - acc: 0.72 - ETA: 10s - loss: 0.8019 - acc: 0.71 - ETA: 10s - loss: 0.8420 - acc: 0.70 - ETA: 10s - loss: 0.8324 - acc: 0.70 - ETA: 9s - loss: 0.8227 - acc: 0.7140 - ETA: 9s - loss: 0.8296 - acc: 0.718 - ETA: 9s - loss: 0.8275 - acc: 0.720 - ETA: 9s - loss: 0.8343 - acc: 0.716 - ETA: 9s - loss: 0.8228 - acc: 0.722 - ETA: 9s - loss: 0.8093 - acc: 0.724 - ETA: 9s - loss: 0.8110 - acc: 0.725 - ETA: 8s - loss: 0.8103 - acc: 0.724 - ETA: 8s - loss: 0.7920 - acc: 0.731 - ETA: 8s - loss: 0.7994 - acc: 0.727 - ETA: 8s - loss: 0.8025 - acc: 0.727 - ETA: 8s - loss: 0.8026 - acc: 0.726 - ETA: 8s - loss: 0.8155 - acc: 0.722 - ETA: 8s - loss: 0.8212 - acc: 0.721 - ETA: 8s - loss: 0.8226 - acc: 0.720 - ETA: 8s - loss: 0.8282 - acc: 0.716 - ETA:

7895/7895 [==============================] - ETA: 7s - loss: 0.8885 - acc: 0.700 - ETA: 8s - loss: 0.7989 - acc: 0.740 - ETA: 8s - loss: 0.6578 - acc: 0.780 - ETA: 8s - loss: 0.6610 - acc: 0.795 - ETA: 8s - loss: 0.6669 - acc: 0.784 - ETA: 8s - loss: 0.6911 - acc: 0.776 - ETA: 8s - loss: 0.7087 - acc: 0.765 - ETA: 8s - loss: 0.6767 - acc: 0.785 - ETA: 8s - loss: 0.6534 - acc: 0.795 - ETA: 8s - loss: 0.6570 - acc: 0.798 - ETA: 8s - loss: 0.6414 - acc: 0.800 - ETA: 8s - loss: 0.6219 - acc: 0.806 - ETA: 8s - loss: 0.6313 - acc: 0.798 - ETA: 8s - loss: 0.6494 - acc: 0.790 - ETA: 8s - loss: 0.6490 - acc: 0.785 - ETA: 8s - loss: 0.6543 - acc: 0.783 - ETA: 8s - loss: 0.6471 - acc: 0.787 - ETA: 8s - loss: 0.6357 - acc: 0.792 - ETA: 8s - loss: 0.6469 - acc: 0.786 - ETA: 8s - loss: 0.6568 - acc: 0.786 - ETA: 8s - loss: 0.6507 - acc: 0.787 - ETA: 8s - loss: 0.6462 - acc: 0.790 - ETA: 7s - loss: 0.6620 - acc: 0.785 - ETA: 7s - loss: 0.6661 - acc: 0.785 - ETA: 7s - loss: 0.6760 - acc: 0.782 - ETA: 

...

7895/7895 [==============================] - ETA: 7s - loss: 0.3040 - acc: 0.860 - ETA: 8s - loss: 0.2707 - acc: 0.900 - ETA: 8s - loss: 0.2926 - acc: 0.906 - ETA: 8s - loss: 0.3462 - acc: 0.880 - ETA: 8s - loss: 0.3479 - acc: 0.888 - ETA: 7s - loss: 0.3255 - acc: 0.893 - ETA: 7s - loss: 0.3054 - acc: 0.897 - ETA: 7s - loss: 0.2923 - acc: 0.905 - ETA: 7s - loss: 0.3041 - acc: 0.904 - ETA: 7s - loss: 0.3112 - acc: 0.900 - ETA: 7s - loss: 0.3114 - acc: 0.901 - ETA: 7s - loss: 0.3084 - acc: 0.903 - ETA: 7s - loss: 0.3004 - acc: 0.904 - ETA: 7s - loss: 0.2957 - acc: 0.904 - ETA: 7s - loss: 0.2965 - acc: 0.904 - ETA: 7s - loss: 0.2908 - acc: 0.906 - ETA: 7s - loss: 0.2956 - acc: 0.904 - ETA: 7s - loss: 0.2939 - acc: 0.905 - ETA: 7s - loss: 0.2902 - acc: 0.906 - ETA: 7s - loss: 0.2895 - acc: 0.908 - ETA: 7s - loss: 0.2843 - acc: 0.908 - ETA: 7s - loss: 0.2835 - acc: 0.909 - ETA: 7s - loss: 0.2819 - acc: 0.908 - ETA: 7s - loss: 0.2815 - acc: 0.907 - ETA: 7s - loss: 0.2835 - acc: 0.907 - ETA: 

7895/7895 [==============================] - ETA: 8s - loss: 0.1372 - acc: 0.980 - ETA: 8s - loss: 0.1923 - acc: 0.930 - ETA: 8s - loss: 0.1847 - acc: 0.933 - ETA: 8s - loss: 0.2086 - acc: 0.915 - ETA: 9s - loss: 0.1992 - acc: 0.924 - ETA: 9s - loss: 0.2581 - acc: 0.926 - ETA: 8s - loss: 0.2606 - acc: 0.917 - ETA: 8s - loss: 0.2818 - acc: 0.907 - ETA: 8s - loss: 0.2691 - acc: 0.915 - ETA: 8s - loss: 0.2831 - acc: 0.916 - ETA: 8s - loss: 0.3084 - acc: 0.909 - ETA: 8s - loss: 0.3107 - acc: 0.906 - ETA: 8s - loss: 0.3053 - acc: 0.906 - ETA: 8s - loss: 0.2982 - acc: 0.907 - ETA: 8s - loss: 0.2997 - acc: 0.906 - ETA: 8s - loss: 0.2932 - acc: 0.907 - ETA: 8s - loss: 0.3272 - acc: 0.901 - ETA: 8s - loss: 0.3189 - acc: 0.904 - ETA: 8s - loss: 0.3131 - acc: 0.907 - ETA: 8s - loss: 0.3086 - acc: 0.908 - ETA: 8s - loss: 0.3052 - acc: 0.909 - ETA: 8s - loss: 0.3087 - acc: 0.909 - ETA: 8s - loss: 0.3174 - acc: 0.907 - ETA: 8s - loss: 0.3332 - acc: 0.903 - ETA: 7s - loss: 0.3279 - acc: 0.904 - ETA: 

7895/7895 [==============================] - ETA: 9s - loss: 0.2599 - acc: 0.900 - ETA: 9s - loss: 0.2187 - acc: 0.930 - ETA: 9s - loss: 0.2441 - acc: 0.926 - ETA: 9s - loss: 0.2639 - acc: 0.925 - ETA: 9s - loss: 0.2727 - acc: 0.924 - ETA: 9s - loss: 0.2722 - acc: 0.923 - ETA: 9s - loss: 0.2869 - acc: 0.922 - ETA: 9s - loss: 0.3098 - acc: 0.920 - ETA: 9s - loss: 0.3173 - acc: 0.915 - ETA: 8s - loss: 0.3065 - acc: 0.916 - ETA: 8s - loss: 0.3010 - acc: 0.912 - ETA: 8s - loss: 0.3057 - acc: 0.911 - ETA: 8s - loss: 0.2956 - acc: 0.912 - ETA: 8s - loss: 0.2988 - acc: 0.905 - ETA: 8s - loss: 0.2972 - acc: 0.906 - ETA: 8s - loss: 0.2952 - acc: 0.907 - ETA: 8s - loss: 0.2937 - acc: 0.905 - ETA: 8s - loss: 0.2965 - acc: 0.905 - ETA: 8s - loss: 0.2913 - acc: 0.908 - ETA: 8s - loss: 0.2919 - acc: 0.908 - ETA: 8s - loss: 0.2920 - acc: 0.906 - ETA: 8s - loss: 0.2846 - acc: 0.909 - ETA: 8s - loss: 0.2843 - acc: 0.907 - ETA: 8s - loss: 0.2923 - acc: 0.905 - ETA: 8s - loss: 0.3002 - acc: 0.904 - ETA: 

7895/7895 [==============================] - ETA: 8s - loss: 0.2082 - acc: 0.940 - ETA: 8s - loss: 0.2177 - acc: 0.940 - ETA: 8s - loss: 0.2370 - acc: 0.926 - ETA: 8s - loss: 0.1992 - acc: 0.940 - ETA: 8s - loss: 0.2177 - acc: 0.936 - ETA: 8s - loss: 0.2058 - acc: 0.940 - ETA: 8s - loss: 0.1911 - acc: 0.945 - ETA: 8s - loss: 0.1870 - acc: 0.942 - ETA: 8s - loss: 0.2345 - acc: 0.937 - ETA: 8s - loss: 0.2339 - acc: 0.936 - ETA: 8s - loss: 0.2462 - acc: 0.927 - ETA: 8s - loss: 0.2384 - acc: 0.930 - ETA: 8s - loss: 0.2436 - acc: 0.929 - ETA: 8s - loss: 0.2382 - acc: 0.930 - ETA: 8s - loss: 0.2661 - acc: 0.926 - ETA: 8s - loss: 0.2569 - acc: 0.930 - ETA: 8s - loss: 0.2476 - acc: 0.931 - ETA: 8s - loss: 0.2453 - acc: 0.930 - ETA: 7s - loss: 0.2447 - acc: 0.930 - ETA: 7s - loss: 0.2433 - acc: 0.930 - ETA: 7s - loss: 0.2389 - acc: 0.930 - ETA: 7s - loss: 0.2395 - acc: 0.930 - ETA: 7s - loss: 0.2388 - acc: 0.927 - ETA: 7s - loss: 0.2390 - acc: 0.927 - ETA: 7s - loss: 0.2359 - acc: 0.928 - ETA: 

7895/7895 [==============================] - ETA: 8s - loss: 0.1730 - acc: 0.940 - ETA: 9s - loss: 0.2091 - acc: 0.950 - ETA: 9s - loss: 0.2498 - acc: 0.933 - ETA: 8s - loss: 0.2586 - acc: 0.930 - ETA: 8s - loss: 0.2542 - acc: 0.928 - ETA: 8s - loss: 0.2363 - acc: 0.930 - ETA: 8s - loss: 0.2210 - acc: 0.931 - ETA: 8s - loss: 0.2059 - acc: 0.937 - ETA: 8s - loss: 0.2158 - acc: 0.926 - ETA: 8s - loss: 0.2120 - acc: 0.922 - ETA: 8s - loss: 0.2085 - acc: 0.921 - ETA: 8s - loss: 0.2138 - acc: 0.921 - ETA: 8s - loss: 0.2055 - acc: 0.924 - ETA: 8s - loss: 0.2210 - acc: 0.918 - ETA: 8s - loss: 0.2121 - acc: 0.921 - ETA: 8s - loss: 0.2134 - acc: 0.920 - ETA: 8s - loss: 0.2265 - acc: 0.916 - ETA: 8s - loss: 0.2241 - acc: 0.917 - ETA: 8s - loss: 0.2263 - acc: 0.917 - ETA: 8s - loss: 0.2251 - acc: 0.918 - ETA: 8s - loss: 0.2279 - acc: 0.917 - ETA: 7s - loss: 0.2310 - acc: 0.917 - ETA: 7s - loss: 0.2332 - acc: 0.916 - ETA: 7s - loss: 0.2269 - acc: 0.918 - ETA: 7s - loss: 0.2202 - acc: 0.920 - ETA: 

In [27]:
model.predict(x_test)

array([[5.6021439e-04, 2.7951907e-02, 5.6688478e-03, ..., 2.3047297e-04,
        6.1831804e-04, 6.3877201e-01],
       [1.5760051e-03, 4.3454260e-01, 8.6641274e-03, ..., 3.7560132e-04,
        3.8586105e-03, 4.9932250e-01],
       [1.5132678e-05, 9.9525499e-01, 8.3858877e-06, ..., 5.3623349e-05,
        2.6701173e-05, 1.5764206e-03],
       ...,
       [2.0870543e-04, 1.4701377e-03, 9.5133355e-06, ..., 3.9015079e-01,
        2.3219709e-06, 1.6406483e-03],
       [2.0483573e-04, 1.6123628e-03, 1.2258000e-05, ..., 3.1655222e-01,
        2.0530488e-06, 1.0922155e-03],
       [2.0027342e-06, 2.4529052e-06, 3.4572258e-09, ..., 9.8936266e-01,
        9.8582011e-11, 4.5715882e-07]], dtype=float32)

In [29]:
#train and test loss and scores respectively
train_loss_score=model.evaluate(x_train,y_train)
test_loss_score=model.evaluate(x_test,y_test)
print(train_loss_score)
print(test_loss_score)

837/837 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - 0s 211us/step
[0.08407034701952429, 0.9727675743311398]
[1.0402508126649663, 0.7180406214088522]


In [30]:
model.save("urbanSound.h5")